In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_5628\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_5628\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.tail()

,Soil Type,Crop Type,Fertilizer Name
749995,1,3,4
749996,2,8,2
749997,4,2,0
749998,3,1,3
749999,2,1,6


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]].values

In [15]:
y = df_train[["Fertilizer Name"]].squeeze()

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np

clf = TabNetClassifier(
    n_d=32, n_a=32, n_steps=5,
    gamma=1.5, lambda_sparse=1e-4,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax',
    verbose=10
)

# Train
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy'],
    max_epochs=1,
    patience=10,
    batch_size=256,
    virtual_batch_size=128
)

# Evaluate
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

# Feature importance

preds = clf.predict(X_train)

KeyboardInterrupt: 

In [28]:
from pytorch_tabnet.tab_model import TabNetClassifier
clf = TabNetClassifier(verbose=1, seed=42, device_name='cpu')  # CPU for clarity

C:\Users\pkitt\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [40]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

AttributeError: 'TabNetClassifier' object has no attribute 'network'

In [89]:
import matplotlib.pyplot as plt

feature_importances = clf.feature_importances_
plt.barh(X.columns, feature_importances)
plt.title("TabNet Feature Importance")
plt.show()

AttributeError: 'TabNetClassifier' object has no attribute 'feature_importances_'

In [29]:

from sklearn.base import BaseEstimator, RegressorMixin
from pytorch_tabnet.tab_model import TabNetRegressor

class TabNetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.model = TabNetRegressor(**kwargs)

    def fit(self, X, y):
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        self.model.fit(
            X_train=X, y_train=y,
            eval_set=[(X, y)],
            eval_metric=['rmse'],
            max_epochs=3,
            patience=3,
            batch_size=256,
            virtual_batch_size=128,
        )
        return self

    def predict(self, X):
        return self.model.predict(X).squeeze()


In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
RFR=RandomForestRegressor(n_estimators=3,)

In [22]:
RFR.fit(X_train,y_train)

RandomForestRegressor(n_estimators=3)

In [31]:
# Initialize models


tabnet_model = TabNetWrapper(n_d=32, n_a=32, n_steps=5, gamma=1.5, lambda_sparse=1e-4, mask_type='entmax')
rf_model = RandomForestRegressor(n_estimators=3, random_state=42)

In [32]:
# Voting ensemble
from sklearn.ensemble import VotingRegressor, RandomForestRegressor

tabnet_model = TabNetWrapper(n_d=32, n_a=32, n_steps=5, gamma=1.5, lambda_sparse=1e-4, mask_type='entmax')
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

ensemble = VotingRegressor(
    estimators=[('tabnet', tabnet_model), ('rf', rf_model)]
)

In [96]:
import numpy as np

# Guarantee NumPy format and 2D shape
y_train = np.asarray(y_train).reshape(-1, 1)

In [33]:
ensemble.fit(X_train, y_train)

C:\Users\pkitt\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.90857 | val_0_rmse: 1.96743 |  0:02:27s
epoch 1  | loss: 3.88101 | val_0_rmse: 1.96819 |  0:05:05s
epoch 2  | loss: 3.87817 | val_0_rmse: 1.97402 |  1:29:41s
Stop training because you reached max_epochs = 3 with best_epoch = 0 and best_val_0_rmse = 1.96743


C:\Users\pkitt\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


VotingRegressor(estimators=[('tabnet', TabNetWrapper()),
                            ('rf', RandomForestRegressor(random_state=42))])

In [36]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.15
